# Automatic differentiation with Zygote
- What is automatic differentiation (AD)
- How to use chain rule to compute compilcated gradients
- How Zygote implements this using so called adjoints
- How to design your own adjoint
    + How to check if it is correctly computed
    + Some advanced examples

In [13]:
using Zygote

## Simple example
Let's start with a simple scalar function of one variable $f(x) = 5x + 3$.

In [18]:
f(x) = 5x + 3;

The exact result of derivative with respect to `x` is of course simple constant function $\frac{df}{dx} = 5$.

In [17]:
df(x) = 5;

In [ ]:
Giving us the expected results

In [19]:
f(10), df(10)

(53, 5)

Our goal is to compute *exact* derivatives of scalar functions (mostly) in an automatic way.

In this simple case Zygote alows us to use the adjoint/transpose operator `'` to compute the same derivative but automatically.

In [20]:
f(10), f'(10)

(53, 5)

The important thing is that Zygote does **source to source** AD, where the derivatives itself are automatically generated functions and are treated by the compiler the same way as if you were to write them yourselves.

In [7]:
@which f'(10)

(::Zygote.var"#39#40")(x) in Zygote at /home/honza/.julia/packages/Zygote/1GXzF/src/compiler/interface.jl:57

In [22]:
@code_typed f'(10)

CodeInfo(
1 ─      goto #3
2 ─      $(Expr(:meta, :inline))
3 ┄      goto #4
4 ─      goto #5
5 ─ %5 = Base.mul_int(5, 1)::Int64
└──      goto #7
6 ─      $(Expr(:meta, :inline))
7 ┄      goto #8
8 ─      goto #9
9 ─      return %5
) => Int64

Looks a little bit more complicated, but when we print the code in machine form we see that this all boils down to a function returning `5`.

In [25]:
@code_native f'(10)

	.text
; ┌ @ interface.jl:57 within `#39'
	movl	$5, %eax
	retq
	nopw	%cs:(%rax,%rax)
; └


Though the notation of `'` operator overloading is neat, it works only for the simplest scalar functions of single variable.

In [36]:
g(x,y) = 2x + 3y

g (generic function with 1 method)

In [38]:
g'(1,1)

MethodError: MethodError: no method matching (::Zygote.var"#39#40"{typeof(g)})(::Int64, ::Int64)
Closest candidates are:
  #39(::Any) at /home/honza/.julia/packages/Zygote/1GXzF/src/compiler/interface.jl:57

Once we move to functions of more variable the thing we want to compute is `gradient`, which returns the derivative with respect to each of the input arguments.

In [41]:
gradient(g, 1, 1)

(2, 3)

In [40]:
@code_typed gradient(g, 1, 1)

CodeInfo(
1 ─      goto #3
2 ─      $(Expr(:meta, :inline))
3 ┄      goto #4
4 ─      goto #5
5 ─ %5 = Base.mul_int(3, 1)::Int64
│   %6 = Base.mul_int(2, 1)::Int64
└──      goto #7
6 ─      $(Expr(:meta, :inline))
7 ┄ %9 = Core.tuple(%6, %5)::Tuple{Int64,Int64}
└──      goto #8
8 ─      return %9
) => Tuple{Int64,Int64}

Which also works on the previous example with function `f`, but instead of returning single number it gives a tuple.

In [42]:
gradient(f, 10)

(5,)

Let's compute the backpropagation for a single layer neural network
$$
n(x) = Wx + b
$$

In [45]:
W, b = rand(2, 3), rand(2);
net(x) = W*x .+ b;
agg(n) = sum(n); # simple aggregation

In [44]:
net([1,3,2])

2-element Array{Float64,1}:
 1.024699723026722
 3.078770609798341

By default the gradient is computed with respect to the input variables.

In [47]:
gradient((x) -> agg(net(x)), [1,2,3])

([1.6579457323925992, 1.4986819280476997, 0.898998435768311],)

If you write it down this should correspont to taking the sum of rows of the weight matrix `W`.

In [50]:
sum(W,dims=1)

1×3 Array{Float64,2}:
 1.65795  1.49868  0.898998

In the field of machine learning we are however usually interested in taking derivatives with respect to the weights and biases. For that Zygote uses the concept of `Params` parameters, which are in turn used in the Flux library.

In [51]:
p = Params([W, b])

Params([[0.8242805451868016 0.6935345698171789 0.3575917782149711; 0.8336651872057976 0.8051473582305209 0.5414066575533398], [0.12011868865849706, 0.28705432802353603]])

Running the gradient instead, returns a dictionary with entries for each paramter in `p`.

In [55]:
grads = gradient(() -> agg(net([1,2,3])), p)

Grads(...)

Gradient dictionary can be indexed directly with the parameter we want the gradient to.

In [57]:
grads[W]

2×3 Array{Float64,2}:
 1.0  2.0  3.0
 1.0  2.0  3.0

Notice that the gradient is a matrix, even though one would expect it to be vector as you were though in calculus. This representation of gradient is more convenient as the update step of gradient descent algortihm looks really simple.

In [59]:
α = 0.01
W .-= α.*grads[W]

2×3 Array{Float64,2}:
 0.814281  0.673535  0.327592
 0.823665  0.785147  0.511407

But keep in mind that from calculus point of view we are still working with scalar functions of multiple variables 
$$
W = 
\begin{pmatrix}
W_{11} & W_{12} & W_{13}\\
W_{21} & W_{22} & W_{23}
\end{pmatrix} \\
b = 
\begin{pmatrix}
b_{1} \\
b_{2}
\end{pmatrix}
$$
and gradients should be in a vector form

In [62]:
vec(grads[W])

6-element Array{Float64,1}:
 1.0
 1.0
 2.0
 2.0
 3.0
 3.0

In [63]:
vec(grads[b])

2-element FillArrays.Fill{Float64,1,Tuple{Base.OneTo{Int64}}}:
 1.0
 1.0

## Under the hood (forward/backward)

Let's look now at how this automatic differentiation works internally. Let's define more complicated nested function such as
$$
s(x) = 5sin(x^2)
$$

In [67]:
s(x) = sin(x^2) * 5

s (generic function with 1 method)

I would like to point out that there is really no much difference from how Zygote computes the derivatives and how you may go about it. What we do is applying the following basic rules to compute the result.
$$
\begin{align}
\frac{d}{dx} x &= 1 \\
\frac{d}{dx} (-u) &= - \frac{du}{dx} \\
\frac{d}{dx} (u + v) &= \frac{du}{dx} + \frac{dv}{dx} \\
\frac{d}{dx} (uv) &= v \frac{du}{dx} + u \frac{dv}{dx} \\
\frac{d}{dx} (u / v) &= (v \frac{du}{dx} - u \frac{dv}{dx}) / v^2 \\
\frac{d}{dx} u^n &= n u^{n-1} \\
\frac{d}{dx} u(v) &= \frac{du}{dv} \frac{dv}{dx} \\
\vdots
\end{align}
$$

The most important thing to understand is that due to Julia's superior meta language capabilities, it is able to understand the code of a function `s` and decompose it in order to apply these rules by pattern matching. Result of which is so called Wengert list of `s`
```julia
y1 = x ^ 2
y2 = sin(y1)
y3 = y2 * 5
```
In other words, we have been able to produce a list of assigments, each of which we are able to differentiate using the standard calculus toolbox. In order to compute the derivative itself we just have to chain the elementary derivatives $\frac{dy_1}{dx}, \frac{dy_2}{dy_1}, \frac{ds}{dy_2}$ and voila 
$$
\frac{ds}{dx} = \frac{dy_1}{dx} \times \frac{dy_2}{dy_1} \times \frac{ds}{dy_2}
$$

Due to the fact that multiplication is asociative, there are two ways how to compute the derivate. 
We can go either from either evaluating first $\frac{dy_1}{dx} \times \frac{dy_2}{dy_1}$ first, or $\frac{dy_2}{dy_1} \times \frac{ds}{dy_2}$?

It's easier to see the distinction if we think algorithmically. Given some
enormous Wengert list with $n$ instructions, we have two ways to differentiate
it:

### Forward
Start with the known quantity $\frac{dy_0}{dx} = \frac{dx}{dx} = 1$
at the beginning of the list. Look up the derivative for the next instruction
$\frac{dy_{i+1}}{dy_i}$ and multiply out the top, getting $\frac{dy_1}{dx}$,
$\frac{dy_2}{dx}$, ... $\frac{dy_{n-1}}{dx}$, $\frac{dy}{dx}$. Because we
walked forward over the Wengert list, this is called *forward mode*. Each
intermediate derivative $\frac{dy_i}{dx}$ is known as a *perturbation*.

### Reverse
Start with the known quantity $\frac{dy}{dy_n} = \frac{dy}{dy} = 1$
at the end of the list. Look up the derivative for the previous instruction
$\frac{dy_i}{dy_{i-1}}$ and multiply out the bottom, getting
$\frac{dy}{dy_n}$, $\frac{dy}{dy_{n-1}}$, ... $\frac{dy}{dy_1}$,
$\frac{dy}{dx}$. Because we walked in reverse over the Wengert list, this is
called *reverse mode*. Each intermediate derivative $\frac{dy}{dy_i}$ is known
as a *sensitivity*.


Zygote uses the latter method as there are huge performance benefits, when computing derivatives with respect to milions of parameters as is the case with ML models.
- taking derivatives of scalar function with respect to more than one variable requires only one pass
- $\frac{dy_i}{dx}$ is a huge Jacobi matrix (`length(y_i) * length(x)`), whereas $\frac{dy}{dy_i}$ is represented more compactly (`length(y) * length(y_i)`)

## How is it implemented?
Even though the way the Wengert lists are created is really important here we will focus only on the differentiation itself.

Going back to our function `s` and suppose that Zygote has already parsed the function into a Wengert list.

In [84]:
w1(x) = x ^ 2
w2(x) = sin(x)
w3(x) = x * 5.0

function s(x)
    y1 = w1(x)
    y2 = w2(y1)
    y3 = w3(y2)
end

s (generic function with 1 method)

Just as a reference, here is the result and gradient that we want to compute.

In [85]:
s(4.0), gradient(s, 4.0)[1]

(-1.4395165833253265, -38.30637921293538)

In Zygote `gradient` is just a wrapper for internal function called `pullback` (or more precisely even more internal `_pullback` and `Pullback` type), which returns both the value of a given function and something called **adjoint** or **adjoint function** representing the gradient. 

In [86]:
s_val, s_back = pullback(s, 4.0)

(-1.4395165833253265, Zygote.var"#37#38"{typeof(∂(s))}(∂(s)))

What should we pass that adjoint function to get our gradient? For that we need to understand what the adjoint represents.

Looking back to the differentiated Wengert list
$$
\frac{ds}{dx} = \frac{dy_1}{dx} \times \frac{dy_2}{dy_1} \times \frac{dy_3}{dy_2},
$$
we can pull back the adjoint for every other intermediate result in the list, while feeding the interemediate value to the next function in the list as the derivative for `y2` is computed not at `x = 4.0` but at `y1`.

In [87]:
y1_val, y1_back = pullback(w1, 4.0)
y2_val, y2_back = pullback(w2, y1_val)
y3_val, y3_back = pullback(w3, y2_val)

(-1.4395165833253265, Zygote.var"#37#38"{typeof(∂(w3))}(∂(w3)))

As expected `y3_val == s_val`. In this way we have thus evaluated the function `s` itself, however we got also the adjoint functions for each intermediate result of that Wengert list, which can be composed in **reverse** order to compute the derivative of the whole list.

In [88]:
y1_back(y2_back(y3_back(1.0)[1])[1])[1]

-38.30637921293538

And this is the derivative we were looking for, where the input `1.0` corresponds to the fact that $\frac{dy_3}{dy_3} = 1$ and what the adjoint function is really computing is the multiplication of two subsequent derivatives/gradients in the chain, i.e.

In [90]:
y3_back(1.0)

(5.0,)

computes $\frac{dy_3}{dy_2} \times \frac{dy_3}{dy_3} = \frac{ds}{dy_2}$, this result is plug again into the pullback of `y2` to compute $\frac{dy_2}{dy_1} \times \frac{dy_3}{dy_2} = \frac{ds}{dy_1}$ and so on until the derivative is computed.

In [91]:
y2_back(y3_back(1.0)[1])

(-4.788297401616923,)

In [92]:
y1_back(y2_back(y3_back(1.0)[1])[1])[1]

-38.30637921293538

The adjoint functions are thus not derivatives/gradients themselves, but they implemented as the multiplication of two derivatives/gradients in the chain. In other words, given a derivative with respect to it's output, they return derivative with respect to it's inputs.

In case of scalar functions of one variable, this is all there is to know about how the adjoint works, however things get a little messy, when we start working with vector functions of multiple variables. such as the following example of composition of functions.

$$
\begin{align}
f(x) &= 
\begin{pmatrix}
{x_{1} + x_{2}} \\
{x_{1}x_{2}}
\end{pmatrix} \\
g(x) &= 
\begin{pmatrix}
e^{(x_{1} + x_{2})} \\
{\ln(x_{1}) + \ln(x_{2})}
\end{pmatrix} \\
h(x) &= g\left(f(x)\right)
\end{align}
$$
For comparison I have precomputed the Jacobi matrices for each of the function.

In [93]:
f(x) = [x[1] + x[2], x[1]*x[2]]
df(x) = [1.0 1.0; x[2] x[1]];

g(y) = [exp(y[1] + y[2]), log(y[1])+log(y[2])]
dg(y) = [exp(y[1] + y[2]) exp(y[1] + y[2]); 1/y[1] 1/y[2]];

In [94]:
h(x) = g(f(x))

h (generic function with 1 method)

For the derivative of $h$ we use the chain rule
$$
h'(x) = g'\left(f(x)\right)f'(x),
$$
where $g'$ and $f'$ are the corresponding Jacobi matrices.

In [95]:
dh(x) = dg(f(x))*df(x)

dh (generic function with 1 method)

What we expect to compute in this case when we ask Zygote to compute pullback of vector functions?

In [100]:
x_val = [1.0, 2.0]
f_val, f_back = pullback(f, x_val)

([3.0, 2.0], Zygote.var"#37#38"{typeof(∂(f))}(∂(f)))

In [101]:
df(x_val)

2×2 Array{Float64,2}:
 1.0  1.0
 2.0  1.0

As we are working with vector functions of multiple variables, we the input for adjoint corresponds to either if we are computing the derivative of $f_1$ 
$$
\nabla_x f_1 =  
\begin{pmatrix}
\frac{\partial f_1}{\partial x_1} &
\frac{\partial f_1}{\partial x_2}
\end{pmatrix}
$$

In [102]:
f_back([1.0, 0.0])

([1.0, 1.0],)

or $f_2$
$$
\nabla_x f_2 =  
\begin{pmatrix}
\frac{\partial f_2}{\partial x_1} &
\frac{\partial f_2}{\partial x_2}
\end{pmatrix}
$$

In [103]:
f_back([0.0, 1.0])

([2.0, 1.0],)

The adjoint function now corresponds to left multiplication the Jacobi matrix with the gradient to it's output.
$$
\nabla \cdot f'
$$
( in the code of adjoints there is usually additional transposition of ∇ to a row vector)

In order to obtain the whole Jacobi matrix of a vector function we have to evaluate the reverse step multiple times, however this is usually not necessary in ML applications. Now back to the function `h`, whose Wenger list representation would look like this:

In [ ]:
function h(x)
	y1 = f(x)
	y2 = g(y1)
end

In [104]:
y1_val, y1_back = pullback(f, x_val)
y2_val, y2_back = pullback(g, y1_val)

([148.4131591025766, 1.791759469228055], Zygote.var"#37#38"{typeof(∂(g))}(∂(g)))

This should correspond to running

In [105]:
h_val, h_back = pullback(h, x_val)

([148.4131591025766, 1.791759469228055], Zygote.var"#37#38"{typeof(∂(h))}(∂(h)))

Choosing to compute $\nabla_x f_2$, we get three time the same result

In [106]:
h_back([0.0, 1.0])

([1.3333333333333333, 0.8333333333333333],)

In [108]:
y1_back(y2_back([0.0, 1.0])[1])

([1.3333333333333333, 0.8333333333333333],)

In [109]:
dh(x_val)[2,:]

2-element Array{Float64,1}:
 1.3333333333333333
 0.8333333333333333

## Footnote: Matrix valued functions
Now that we know how Zygote works on vector functions, let's complicate the things a little bit for ourselves with matrix-matrix valued functions such as the linear layer of neural network from the beggining. 

In [112]:
W, b = rand(2, 3), rand(2);
n(X) = W*X .+ b

n (generic function with 1 method)

`X` is now a matrix, representing a batch of samples.

In [110]:
X_val = rand(3, 10)

3×10 Array{Float64,2}:
 0.329415    0.461212  0.13606   0.0893104  …  0.807367  0.203848  0.531409
 0.00551086  0.565459  0.957635  0.362832      0.656018  0.822869  0.297294
 0.919522    0.617153  0.829805  0.740371      0.808664  0.76231   0.90353

Even though that is usually not the case, let's try to compute the derivative of $n$ with respect to the matrix of values $X$.

In [113]:
n_val, n_back = pullback(n, X_val)

([1.8745972125649726 1.7936400679733917 … 1.7180494740487249 2.064463624172422; 0.8881247845229077 0.8233833186720201 … 0.7403741536734852 1.0110236041265654], Zygote.var"#37#38"{typeof(∂(n))}(∂(n)))

As expected the output another matrix of size 2x10

In [115]:
n_val

2×10 Array{Float64,2}:
 1.8746    1.79364   1.72908   1.55523  …  2.25938  1.71805   2.06446
 0.888125  0.823383  0.734574  0.65036     1.14072  0.740374  1.01102

Now imagine function `n` to be a part of big model with some scalar loss function, whose derivative we compute in reverse mode. When we get to it we have already collected some gradient with respect to the output of `n`, which has the same size as `n_val` (rememmber the `vec` transformation of matrices to vector is just a matter of conveniece), therefore the input for our adjoint function `n_back` will have the same dimensions.

In [118]:
Δ = rand(size(n_val)...)

2×10 Array{Float64,2}:
 0.611034  0.271175  0.283491  0.947586  …  0.906588  0.271459  0.948848
 0.314994  0.891307  0.96543   0.757883     0.235232  0.328397  0.490567

Giving us the same dimensions as the input of the matrix function.

In [120]:
n_back(Δ)[1]

3×10 Array{Float64,2}:
 0.723871   0.793104   0.850247   1.29172  …  0.928261   0.439914   1.12496
 0.0657984  0.0403055  0.0426331  0.10602     0.0941948  0.0320163  0.102196
 0.668416   0.683785   0.732171   1.17535     0.872146   0.394035   1.03869

In calculus however we are usually not taught how to differentiate such functions, as they are just hidden vector functions of multiple variables, whose Jacobi matrices are computed in the same way, however making sense of it is may not be as straightforward for everyone. 

Take for example the matrix-matrix multiplication in the function `n`. Suppose there is some projection from matrix to vector (in julia this is the `vec` function which gives the matrix in column major order) giving us vectors $n_v$, $x_v$.
$$
n_v(x_v) = n_v
\begin{pmatrix}
x_{1,1} \\
\vdots  \\
x_{3,10}
\end{pmatrix}
= 
\begin{pmatrix}
n_{1,1} \\
\vdots  \\
n_{2,10}
\end{pmatrix}
$$
Now we can compute the jacobian for such "vectorized" function
$$
n_v'(x_v) = W \otimes I_{3},
$$
where $I_{3}$ is identity 3x3 and $\otimes$ is Kronecker/tensor product of matrices. This may look complicated but in reality, when we multiply this from the left with the "vectorized" gradient $\Delta_v$, we get that the adjoint operation boils down to a simple matrix multiplication of
$$
\Delta^TW
$$

It is often the case, that the Jacobi matrix of "vectorized" function does have a nice structure, that allows us to write the adjoints in a more effective way instead of computing the whole Jacobi matrix and doing the multiplication. 

## How to write your own custom adjoints
Though the automatic differentiation in Zygote is really powerfull tool, sometimes you will be forced to write your own adjoint functions. The reasons for this may be 
- performance (functions writte in a way that Zygote understands it may not be most performant solution)
- unsupported operations (array mutation)
- replacing some operations when computing reverse (argmax <-> softmax)
- bragging rights

The users are encouraged to write their own adjoints in order to get the most out of this great tool.

In [122]:
using Zygote: @adjoint

### Simple examples
Let's start again with some simple scalar function of two variables.

In [124]:
add(a, b) = a + b

add (generic function with 2 methods)

Adjoints are defined using the `@adjoint` macro, which is a shorthand for defining `pullback(typeof(:add), a, b)`.

In [125]:
@adjoint add(a, b) = add(a, b), Δ -> (Δ, Δ)

The first element of the tuple allows us to redefine function `add`, though here we just call the function itself. The second element is the adjoint function itself, which in this case is really simple, returning the gradient/derivative multiplied by one for each of the inputs.

In [140]:
add_val, add_back = pullback(add, 2, 3)
add_back(1)

I'll be back!?


(1, 1)

In situation where it may be useful, we can redefine the function itself just for the purposes of AD. This can be usefull in cases, where the code of the adjoint and the function itself is similar and we would like to reuse some of the intermediate results.

In [139]:
@adjoint add(a, b) = begin
    println("I'll be back!?")
    add(a, b) 
end, Δ -> (Δ, Δ)

Note that the original function defintion is kept untouched 

In [142]:
add(2, 3)

5

### Gradcheck
How do we check if the adjoint we have defined is correct?
- Zygote itself (only if the automatic pullback works)
- numerical derivatives
- by hand

### Caveats (global variables)
If there are some global variables used in the function or if we use a type as a functor (calling the type itself), which is often the case with neural network modules.

In [137]:
function addg(a) 
    let b = 3;
    a + b
end

LoadError: syntax: incomplete: "function" at none:1 requires end

In [134]:
addg_val, addg_back = Zygote._pullback(addg, 4)

(7, ∂(addg))

In [135]:
addg_back(2)

(nothing, 2)

In [ ]:
@adjoint add(a, b) = add(a, b), Δ -> (Δ)

### Best practices